In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print('device avaliable : ',device_name)
with tf.device(device_name):
  pass

In [ ]:
import os, keras, numpy
import matplotlib.pyplot as plt
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.datasets import mnist
# from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Concatenate

In [ ]:
def define_discriminator(in_shape=(28,28,1), n_classes=10):

  in_label = Input(shape=(1,), name = "input_label")
  li = Embedding(n_classes, 50)(in_label)
  li = Dense(in_shape[0] * in_shape[1])(li)
  li = Reshape((in_shape[0], in_shape[1], 1))(li)

  in_image = Input(shape=in_shape, name = "input_image")

  merge = Concatenate()([in_image, li])

  fe = Conv2D(128, (3,3), strides=(2,2), padding='same', activation = 'relu')(merge)
  # fe = LeakyReLU(alpha=0.2)(fe)
  fe = Conv2D(128, (3,3), strides=(2,2), padding='same', activation = 'relu')(fe)
  # fe = LeakyReLU(alpha=0.2)(fe)
  fe = Flatten()(fe)
  fe = Dropout(0.2)(fe)
  out_layer = Dense(1, activation='sigmoid')(fe)

  model = Model([in_image, in_label], out_layer)

  opt = Adam(learning_rate= 3e-4, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model
a = define_discriminator()
a.summary()
#keras.utils.plot_model(a)

In [ ]:
def define_generator(latent_dim = 100, n_classes=10):

  in_label = Input(shape=(1,))
  li = Embedding(n_classes, 50)(in_label)
  li = Dense(7 * 7)(li)
  li = Reshape((7, 7, 1))(li)

  in_lat = Input(shape=(latent_dim,))

  n_nodes = 128 * 7 * 7
  gen = Dense(n_nodes, activation = 'relu')(in_lat)
  # gen = LeakyReLU(alpha=0.2)(gen)
  gen = Reshape((7, 7, 128))(gen)

  merge = Concatenate()([gen, li])

  gen = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', activation = 'relu')(merge)
  # gen = LeakyReLU(alpha=0.2)(gen)
  gen = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', activation = 'relu')(gen)
  # gen = LeakyReLU(alpha=0.2)(gen)
  # gen = Conv2DTranspose(128, (3,3), strides=(1,1), padding='same', activation = 'relu')(gen)
  out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(gen)

  model = Model([in_lat, in_label], out_layer)
  return model
b = define_generator(100)
b.summary()
#keras.utils.plot_model(b)

In [ ]:
def define_gan(g_model, d_model):

  d_model.trainable = False

  gen_noise, gen_label = g_model.input
  gen_output = g_model.output

  gan_output = d_model([gen_output, gen_label])

  model = Model([gen_noise, gen_label], gan_output)

  opt = Adam(learning_rate= 3e-4, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt,  metrics=['accuracy'])
  
  return model
c = define_gan(b, a)
c.summary()
#keras.utils.plot_model(c)

In [ ]:
def load_real_samples():
  # load dataset
  (trainX, trainy), (_, _) = mnist.load_data()
  """trainX = trainX[:500]
  trainy = trainy[:500]"""
  #print(trainX.shape)
  # expand to 3d, e.g. add channels
  X = expand_dims(trainX, axis=-1)
  #print(X.shape)
  # convert from ints to floats
  X = X.astype('float32')
  # scale from [0,255] to [-1,1]
  X = (X - 127.5) / 127.5
  return [X, trainy]
k = load_real_samples()
k[0].shape,k[1].shape
print(k[0][:16].shape)
def save_plot(x_input,lebel,n=4):
    for i in range(n*n):
        plt.subplot(n, n, i+1)
        plt.title(str(lebel[i]),fontsize=10,color="red",fontweight=10,pad='2.0',backgroundcolor='yellow')
        plt.imshow(x_input[i,:,:,0], cmap = 'gray_r')
        plt.axis('off')
    plt.show()
save_plot(k[0][:16], k[1][:16])

In [ ]:
def generate_real_samples(dataset, n_samples):
	images, labels = dataset
	ix = randint(0, images.shape[0], n_samples)
	X, labels = images[ix], labels[ix]
	y = ones((n_samples, 1))
	return [X, labels], y
d = generate_real_samples(k, 32)
d[0][0].shape, d[0][1].shape, d[1].shape

In [ ]:
def generate_latent_points(latent_dim, n_samples, n_classes=10):
  x_input = randn(latent_dim * n_samples)
  z_input = x_input.reshape(n_samples, latent_dim)
  labels = randint(0, n_classes, n_samples)
  return [z_input, labels]
p = generate_latent_points(100, 32)
print(p[0].shape,p[1].shape)
 

def generate_fake_samples(generator, latent_dim, n_samples):
	z_input, labels_input = generate_latent_points(latent_dim, n_samples)
	images = generator.predict([z_input, labels_input])
	y = zeros((n_samples, 1))
	return [images, labels_input], y
with tf.device(device_name):
  kh = generate_fake_samples(b, 100, 32)
  print(kh[0][0].shape, kh[0][1].shape, kh[1].shape)
  #kh[0][0][0]

In [ ]:
def save_plot(x_input,lebel,n=4):
    for i in range(n*n):
        plt.subplot(n, n, i+1)
        plt.title(str(lebel[i]),fontsize=10,color="red",fontweight=10,pad='2.0',backgroundcolor='yellow')
        plt.imshow(x_input[i,:,:,0], cmap = 'gray_r')
        plt.axis('off')
    plt.show()
def summarize_the_model(generator, latent_dim = 100):
    latent_points, labels = generate_latent_points(latent_dim= 100, n_samples= 16)
    X  = generator.predict([latent_points, labels])
    # scale from [-1,1] to [0,1]
    X = (X + 1) / 2.0
    save_plot(X,labels, n=4)

In [ ]:
def train(g_model, d_model, gan_model, dataset, latent_dim= 100, n_epochs=3, n_batch=128):

  bat_per_epo = int(dataset[0].shape[0] / n_batch)
  print("batch per epoch: ", bat_per_epo)
  half_batch = int(n_batch / 2)
  print("half batch: ", half_batch)

  for i in range(n_epochs):
    d_loss_r,d_loss_f = 0.0,0.0
    g_loss = 0.0
    
    for j in range(bat_per_epo):

      [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
      d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
      d_loss_r += d_loss1

      [X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
      d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)
      d_loss_f += d_loss2

      [z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
      y_gan = ones((n_batch, 1))
      g_loss1,_ = gan_model.train_on_batch([z_input, labels_input], y_gan)
      g_loss += g_loss1
    print('epoch -> %d, d_loss_real = %.4f, d_loss_fake = %.4f, g_loss = %.4f' %(i+1, d_loss_r, d_loss_f, g_loss))
    summarize_the_model(g_model)
    model_json = g_model.to_json()
    if i == 0:
      if "generator.json":
        os.remove("generator.json")
      with open("generator.json","w") as json_file:
        json_file.write(model_json)
    if g_model:
      os.remove("generator.json")
      model_json = g_model.to_json()
      with open("generator.json","w") as json_file:
        json_file.write(model_json)

In [ ]:
with tf.device(device_name):

  latent_dim = 100
  n_epochs = 30
  d_model = define_discriminator()
  g_model = define_generator(latent_dim)
  gan_model = define_gan(g_model, d_model)
  dataset = load_real_samples()
  print('READY TO GO !!!')
  
  train(g_model, d_model, gan_model, dataset, n_epochs, latent_dim)

In [ ]:
# # save the model
# model_json = g_model.to_json()
# with open("generator.json","w") as json_file:
#   json_file.write(model_json)

# load the model
from keras.models import model_from_json
json_file = open("generator.json","r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [ ]:
latent_dim = 100
n_samples = 16
z_input, labels = generate_latent_points(latent_dim, n_samples)
print(z_input.shape, labels.shape)
data = [z_input,labels]
pred = loaded_model.predict(data)
pred = (pred +1 ) / 2.0
print(pred.shape)
save_plot(pred,labels,n=4)

In [ ]:
# latent_dim = 100
# n_samples = 16
# x_input = randn(latent_dim * n_samples)
# print(x_input.shape)
# z_input = x_input.reshape(n_samples, latent_dim)
# print(z_input.shape)
# import numpy as np
# n_classes = 10
# labels = randint(0, n_classes, n_samples)
# print(labels)
# labels = labels.reshape(n_samples,)
# print(labels.shape)
# data = [z_input,labels]
# pred = loaded_model.predict(data)
# pred = (pred +1 ) / 2.0
# print(pred.shape)
# save_plot(pred,labels,n=4)

In [ ]:
# import os
# os.remove("generator.json")